---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [1]:
import pandas as pd

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)

df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [2]:
import re
def date_sorter():


    regex_list = [r'((?:\d{1,2}[-,\.\s]*)?(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[-,\.\s]*(?:\d{1,2},[-,\.\s]*)?\d{4})',
                  r'((?:\d{1,2})[-\s/]*(?:\d{1,2})[-\s/]*(?:\d{2,4}))',
                  r'((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]* (?:\d{1,2} )?\d{4})']
    generic_re = re.compile( '|'.join( regex_list))
    new_df = df.str.extract(generic_re, expand = True)
    new_df.columns = ['dt_1', 'dt_2', 'dt_3']
    new_df.loc[new_df['dt_1'].isnull(),'dt_1'] = new_df['dt_2']
    new_df.loc[new_df['dt_1'].isnull(),'dt_1'] = new_df['dt_3']
    excp_r = ['10-15', '011/14', '708-810', '50-100', 'Janaury 1993', '11-16', '2June, 1999', 'Decemeber 1978', '67 5/2000']
    values = ['6/29/81', '11/14/83', 'August 2008', 'July 1995', 'January 1993', 'Feb 1993', '2 June 1999', 'December 1978', '5/2000']
    excp_dict = dict(zip(excp_r, values))
    new_df['dt_1'] = new_df['dt_1'].replace(excp_dict)
    new_df['dt_1'] = pd.to_datetime(new_df['dt_1'], errors = 'coerce')
    dff = new_df['dt_1'].sort_values().reset_index().drop('dt_1', axis = 1)
    dff = pd.Series(dff['index']).astype(int)
    
    return dff

date_sorter()

0        9
1       84
2        2
3       53
4       28
5      474
6      153
7       13
8      129
9       98
10     111
11     225
12      31
13     171
14     191
15     486
16     335
17     415
18      36
19     405
20     323
21     422
22     375
23     380
24     345
25      57
26     481
27     436
28     104
29     299
      ... 
470    208
471    243
472    139
473    320
474    383
475    286
476    480
477    244
478    431
479    279
480    198
481    463
482    381
483    366
484    255
485    439
486    401
487    475
488    257
489    152
490    235
491    464
492    253
493    231
494    427
495    141
496    186
497    161
498    413
499    490
Name: index, Length: 500, dtype: int64